In [1]:
import pandas as pd
import os
import sys
script_dir = os.getcwd() # Get the current working directory
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

df = pd.read_parquet('../data/transformed/tabular.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [2]:
from datetime import datetime
from src.data_split import  train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 9, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(56330, 674)
y_train.shape=(56330,)
X_test.shape=(31964, 674)
y_test.shape=(31964,)


In [3]:
import numpy as np

class BaselineModelPreviosHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series ) -> None:
        pass

    def predict( self, X_test: pd.DataFrame)-> np.array:
        return X_test['rides_previous_1_hour']

In [4]:
model = BaselineModelPreviosHour()
predictions = model.predict(X_test)
predictions


0         5.0
1        10.0
2        10.0
3        13.0
4        10.0
         ... 
31959     0.0
31960     0.0
31961     0.0
31962     0.0
31963     0.0
Name: rides_previous_1_hour, Length: 31964, dtype: float32

In [5]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:4f}')

test_mae=6.395413


In [6]:
class BaselineModelPreviousWeek:
    """ 
    Prediction = actual demand observed at t-7 days
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series ) -> None:
        pass

    def predict( self, X_test: pd.DataFrame)-> np.array:
        return X_test[f'rides_previous_{7*24}_hour']

    

In [7]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:4f}')



test_mae=4.212739


In [8]:
class BaselineModelLast4Weeks:
    """ 
    Prediction = Average demand observed at t-7 days, t-24 days, t-21 days, and t-28 days
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series ) -> None:
        pass

    def predict( self, X_test: pd.DataFrame)-> np.array:
        return 0.25 * (
            X_test[f'rides_previous_{7*24}_hour'] +
            X_test[f'rides_previous_{2*7*24}_hour'] +
            X_test[f'rides_previous_{3*7*24}_hour'] +
            X_test[f'rides_previous_{4*7*24}_hour']
        )

    

In [9]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:4f}')

test_mae=3.688220
